# train.csv
* 이게 뭔 놈인고 하니 그 유명한(?) 타이타닉 데이터입니다
* 타이타닉 탑승자들의 생존률을 추론해봅시다

* 컬럼 구성 : PassengerId, Survived, Pclass, Name, Sex, Age, SibSp, Parch, Ticket, Fare, Cabin, Embarked(경유지)
* 우리는 Survived, Sex, Age, Embarked 컬럼만 이용할 겁니다
* feature -> Sex, Age, Embarked // label -> Survived ::: 이렇게 구성해서 지도학습을 할 것이고
* 당연히 전처리부터 들어가야하는데 엑셀파일 보면 알겠지만 군데군데 결측치가 존재함 -> 평균값으로 대체합니다
* 문자열에 대한 인코딩도 잊지 말아주시고요 (label encoding을 이용하여 데이터프레임으로 출력)

* 생존률 구하기 과정
* pandas 라이브러리로 csv 가져오기 -> 필요한 컬럼만 남기고 drop 처리 -> fillna()를 이용한 결측치 처리
* -> LabelEncoder를 활용하여 인코딩 처리 수행

* 결측치 : Sex - 남자로 가정, Age - 평균, Embarked - S로 처리

In [144]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import numpy as np
import math

In [130]:
# 그 많은 컬럼들 중에 4개밖에 안 쓴다면 그냥 4개만 남기는 게 더 빠르지 않아?
titanic_df = pd.read_csv("./csv/train.csv")
titanic_df = titanic_df[['Survived','Sex','Age','Embarked']]

In [131]:
age_mean = titanic_df["Age"].mean()
titanic_df['Sex'] = titanic_df['Sex'].fillna("male")
titanic_df['Age'] = titanic_df['Age'].fillna(age_mean).round(0).astype(int)
titanic_df['Embarked'] = titanic_df['Embarked'].fillna("S")

# embarked나 age는 그렇다 쳐도 Sex를 male로 채우는 이유? 아마도 남자가 많아서?

In [132]:
titanic_df.info()

# 컬럼들 중에 Sex, Embarked가 object 타입으로 나오니까 인코딩을 해줍시다

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Survived  891 non-null    int64 
 1   Sex       891 non-null    object
 2   Age       891 non-null    int32 
 3   Embarked  891 non-null    object
dtypes: int32(1), int64(1), object(2)
memory usage: 24.5+ KB


In [133]:
encoder_Sex = LabelEncoder().fit(titanic_df['Sex'])
encoder_Embarked = LabelEncoder().fit(titanic_df['Embarked'])

# 굳이 객체 만들어서 쓰는데 이렇게까지 쓸데없이 변수를 많이 만들 일이야?

In [134]:
# 인코딩 했으니까 컬럼에 대신 넣을 거에요

encoder = LabelEncoder()
df = titanic_df
df['Sex'] = encoder.fit_transform(titanic_df['Sex'])
df['Embarked'] = encoder.fit_transform(titanic_df['Embarked'])

In [143]:
def age_group(x):
    x = math.trunc(x/10)
    if(x == 0): return "Under 10"
    elif(x == 1): return "10"
    elif(x == 2): return "20"
    elif(x == 3): return "30"
    elif(x == 4): return "40"
    elif(x == 5): return "50"
    elif(x == 6): return "60"
    elif(x == 7): return "70"
    elif(x == 8): return "80"
    elif(x == 9): return "90"
    elif(x == 10): return "100"
    else: return "Over 100"

df['Age_group'] = df['Age'].apply(age_group)

df

,Survived,Sex,Age,Embarked,Age_group
0,0,1,22,2,20
1,1,0,38,0,30
2,1,0,26,2,20
3,1,0,35,2,30
4,0,1,35,2,30
...,...,...,...,...,...
886,0,1,27,2,20
887,1,0,19,2,10
888,0,0,30,2,30
889,1,1,26,0,20


In [176]:
# 그룹화 해서 생존률, 사망률 추론 가능하겠네.. 근데 groupby 이거 맞아?

df2 = df.groupby(['Survived','Sex']).count()
df2

Age  Embarked  Age_group
Survived Sex                          
0        0     81        81         81
         1    468       468        468
1        0    233       233        233
         1    109       109        109

In [177]:
# 오히려 이쪽이 더 알아보기에는 쉽지 않나. 아닌가.
pd.get_dummies(df)

,Survived,Sex,Age,Embarked,Age_group_10,Age_group_20,Age_group_30,Age_group_40,Age_group_50,Age_group_60,Age_group_70,Age_group_80,Age_group_Under 10
0,0,1,22,2,0,1,0,0,0,0,0,0,0
1,1,0,38,0,0,0,1,0,0,0,0,0,0
2,1,0,26,2,0,1,0,0,0,0,0,0,0
3,1,0,35,2,0,0,1,0,0,0,0,0,0
4,0,1,35,2,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,1,27,2,0,1,0,0,0,0,0,0,0
887,1,0,19,2,1,0,0,0,0,0,0,0,0
888,0,0,30,2,0,0,1,0,0,0,0,0,0
889,1,1,26,0,0,1,0,0,0,0,0,0,0


In [ ]:
# 타이타닉 성별에 대한 One-Hot Encoder 수행해보기

oh_encoder = OneHotEncoder()
oh_encoder.fit(df['Sex'])
oh_labels = oh_encoder.transform(df['Sex'])